# Scheduling algorithm for weekly clients


In [23]:
import json
import cpmpy as cp
import numpy as np
import time

import pprint

pp = pprint.PrettyPrinter(indent=2)

In [24]:
def ttm(time_str):
    ts = time_str.split(":") if type(time_str) == str else time_str
    return int(ts[0]) * 60 + int(ts[1]) if type(time_str) == str else time_str


def mtt(minutes):
    hours, mins = divmod(minutes, 60)
    return f"{hours:02d}:{mins:02d}"

In [25]:
def read_client_data(file_name, working_days):
    with open(f"data/{file_name}") as file:
        data = json.load(file)
        clients = [client for client in data.keys()]
        duration = [client["duration"] for client in data.values()]
        availability = [
            [
                (
                    ttm(
                        client["availability"].get(
                            day, {"start_time": 0, "end_time": 0}
                        )["start_time"]
                    ),
                    ttm(
                        client["availability"].get(
                            day, {"start_time": 0, "end_time": 0}
                        )["end_time"]
                    ),
                )
                for day in working_days
            ]
            for client in data.values()
        ]

        return clients, availability, duration


working_days = ["Friday", "Saturday"]
file_name = "clients_list.json"
clients, availability, duration = read_client_data(file_name, working_days)

In [26]:
def generate_schedule(clients, availability, duration, working_days, n_scheduled=-1):
    min_time = np.min(availability)
    max_time = np.max(availability)

    scheduled_times = np.array(
        [[cp.intvar(min_time, max_time) for _ in clients] for _ in working_days]
    )
    ind = np.array([cp.boolvar(len(availability)) for _ in working_days])
    model = cp.Model()

    hole_times = []

    for i, client in enumerate(availability):
        model += cp.sum(ind[:, i]) <= 1
        for d, (start, stop) in enumerate(client):
            # basic scheduled or not scheduled filtering
            model += (ind[d][i] == 0).implies(scheduled_times[d][i] == 0)
            model += (ind[d][i] == 1).implies(
                (scheduled_times[d][i] >= start)
                & (scheduled_times[d][i] <= stop - duration[i])
            )

            if start == 0:
                model += ind[d][i] == 0

            if clients[d][i] in ["Žan Garantini", "Tilen Medved"]:
                model += ind[d][i] == 1

            model += (ind[d][i] == 1).implies((scheduled_times[d][i] % 5) == 0)

            for j in range(i + 1, len(clients)):
                model += ((ind[d][i] == 1) & (ind[d][j] == 1)).implies(
                    (scheduled_times[d][i] + duration[i] <= scheduled_times[d][j])
                    | (scheduled_times[d][j] + duration[j] <= scheduled_times[d][i])
                )

        LV = 24 * 60
        min_time = cp.min(scheduled_times[d] * ind[d] + (1 - ind[d]) * LV)
        max_time = cp.max((scheduled_times[d] + duration) * ind[d] - (1 - ind[d]) * LV)
        hole_time = cp.max([max_time - min_time - cp.sum(ind[d] * duration), 0])
        hole_times.append(hole_time)

    w_ind = 1
    w_h = 0.1
    w_e = 0.001

    clients_scheduled = cp.sum(ind)

    waiting_time = cp.sum(hole_times)

    early_start = cp.sum(scheduled_times)

    obj = w_ind * clients_scheduled - w_h * waiting_time + w_e * early_start

    model.maximize(obj)

    return model, scheduled_times, ind, obj

In [27]:
def myprint():
    print([[time.value() for time in day] for day in scheduled_times])

In [28]:
store = []
store_ind = []
obj_value = -1

model, scheduled_times, ind, obj = generate_schedule(
    clients, availability, duration, working_days, n_scheduled=8
)

t_end = time.time() + 300
while time.time() < t_end and model.solve():
    schedule = [[time.value() for time in day] for day in scheduled_times]
    inds = [[item.value() for item in day] for day in ind]
    if schedule not in store:
        store.append(schedule)
        store_ind.append(store_ind)

In [ ]:
def custom_sort(schedule):
    friday_start = min(x for x in schedule[0] if x > 0)
    saturday_start = min(x for x in schedule[1] if x > 0)

    return (friday_start, saturday_start)


sorted_schedules = sorted(store, key=custom_sort)


with open("schedules.txt", "w") as file:
    for schedule in sorted_schedules:
        for d, day in enumerate(schedule):
            for j, time in enumerate(day):
                if time != 0:
                    file.write(f"{mtt(time)}: {clients[j]}\n")

            file.write("---\n")

        file.write("------------------------\n")